In [1]:
import os 
import imp
import warnings
import numpy as np
import pandas as pd
import importlib as imp
import itertools
from tqdm import tqdm, tqdm_notebook
from utils import tools
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

PATH_TO_BENCHMARKS = './/data//'

**Задача**

*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>

In [4]:
txtfiles = [f for f in os.listdir(PATH_TO_BENCHMARKS) if os.path.isfile(os.path.join(PATH_TO_BENCHMARKS, f))]
txtfiles

['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']

In [5]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
machine_part_matrix_sample = np.array([[1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

In [4]:
pd.DataFrame(machine_part_matrix_sample)

,0,1,2,3,4
0,1,0,0,1,0
1,0,1,1,0,1
2,1,0,0,0,0
3,0,1,1,0,0
4,0,0,0,1,0


Построение $similarity\ matrix$ по формуле $$S_{ij} = \frac{a_{ij}}{a_{ij} + b_{ij} + c_{ij}}$$
$a_{ij}$ - кол-во машин, которые обрабатывают детали $i, j$
$b_{ij}$ - кол-во машин, которые обрабатывают деталь $i$, но не $j$
$c_{ij}$ - кол-во машин, которые обрабатывают деталь $j$, но не $i$

In [5]:
pd.DataFrame(similarity_matrix)

NameError: name 'similarity_matrix' is not defined

Матрицу удобнее представить в виде списка: $[[(i, j), S_{i, j}]]$

In [7]:
np_similarity

[[(1, 2), 1.0], [(1, 4), 0.5], [(2, 4), 0.5], [(0, 3), 0.3333333333333333]]

Итерируясь в порядке убывания по списку составленному по $similarity\ matrix$ генерируем первоначальное решение (разбиение на $cells$)

In [6]:
cells_p

NameError: name 'cells_p' is not defined

$Cell 1:$ [1, 2, 4]
$Cell 2:$ [0, 3]

In [7]:
pd.DataFrame(machine_part_matrix_sample)[[1, 2, 4, 0, 3]]

,1,2,4,0,3
0,0,0,0,1,1
1,1,1,1,0,0
2,0,0,0,1,0
3,1,1,0,0,0
4,0,0,0,0,1


In [8]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
cells_p, cells_m = tools.get_solution(machine_part_matrix, 2)

In [26]:
df = tools.solution_df(cells_p, cells_m, machine_part_matrix)
tools.compute_grouping_efficacy(cells_p, cells_m, df)

0.26881720430107525

In [81]:
from algorithm import SimulatedAnnealing

SimulatedAnnealing = imp.reload(SimulatedAnnealing)
params = {
    'initial_temperature' : 10,
    'final_temperature'   : 0.002,
    'chain_len'           : 10,
    'len_of_period'       : 6,
    'numb_of_cells'       : 2,
    'check'               : 10,
    'cooling_rate'        : 0.7,
}

SA = SimulatedAnnealing.SimulatedAnnealing(machine_part_matrix)
SA.set_params(params)
SA.solve()

begin
              f: {'current': 0.26881720430107525, 'neigh': 0.28205128205128205, 'best': 0}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 2, 'optimal': 0}


end
              f: {'current': 0.28205128205128205, 'neigh': 0.28205128205128205, 'best': 0.28205128205128205}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 3, 'optimal': 2}


begin
              f: {'current': 0.3575418994413408, 'neigh': 0.3641304347826087, 'best': 0.28205128205128205}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 3, 'optimal': 2}


end
              f: {'current': 0.3641304347826087, 'neigh': 0.3641304347826087, 'best': 0.3641304347826087}
temperature    : {'current': 10, 'final': 0.002}
number of cells: {'initial': 4, 'optimal': 3}


begin
              f: {'current': 0.33793103448275863, 'neigh': 0.3424657534246575, 'best': 0.3641304347826087}
temperature    : {'current': 10, 'final': 0.002}
number of cells: 

In [17]:
import math
math.exp(0)

1.0

In [25]:
for i in range(3):
    for j in range(4):
        if j > 1:
            break
        print(i, j)

0 0
0 1
1 0
1 1
2 0
2 1
